<a href="https://colab.research.google.com/github/urieliram/statistical/blob/main/Tarea12.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [42]:
import numpy as np
import pandas as pd

In [43]:
df  = pd.read_csv('minitest.csv')
#df1 = df.iloc[:, [1,2,0]]

#df1 = df.loc[df.iloc[:,19].isin(['T'])] ## 'T' = training set
df1 = df.loc[(df['Y']>=3)] & df.loc[(df['X0']==0)]
df1 = df1[['Y']]

#df1 = df.loc[df.iloc[:,19].isin(['T'])] ## 'T' = training set

#df2 = df1['Y']  
#df1 = df1[['X1','X2','X3','X4','X5','X6','X7','X8','X9','X10']] ## Regresores#

X   = pd.DataFrame(df[['X0','X1','X2','X3','X4']]).to_numpy()
Y   = pd.DataFrame(df[['Y']]).to_numpy()

In [44]:
print(type(X))
print(X)

<class 'numpy.ndarray'>
[[ 1  4  4 16 16]
 [ 1 -1 -9  1 81]
 [ 1  6 -2 36  4]
 [ 1 -8  7 64 49]
 [ 1 -2  1  4  1]
 [ 1  7  3 49  9]
 [ 1 -9 -1 81  1]
 [ 1 -8  9 64 81]
 [ 1  4 -7 16 49]]


In [45]:
#https://towardsdatascience.com/numpy-cheat-sheet-4e3858d0ff0e}
X   = pd.DataFrame(df[['X0','X1','X2','X3','X4']]).to_numpy()
Y   = pd.DataFrame(df[['Y']]).to_numpy()
A = np.matmul(X.T, X)  # Multiplica dos matrices
A = np.linalg.inv(A)   # Calcula inversa de A 
B = np.matmul(X.T,Y)
beta = np.matmul(A,B)
prediccion = np.matmul(X,beta)    
print(prediccion)

[[4.54638371]
 [6.0085059 ]
 [6.18866689]
 [3.02076176]
 [3.78154498]
 [5.74605327]
 [4.27525042]
 [2.8380248 ]
 [6.59480827]]


In [46]:
from sklearn.linear_model import LinearRegression

model = LinearRegression()
model.fit(X, Y)
r_sq = model.score(X, Y)
print('coefficient of determination:', r_sq)
print('intercept:', model.intercept_)
intercept: 5.633333333333329
print('slope:', model.coef_)
y_pred = model.predict(X)
print('predicted response:', y_pred, sep='\n')

coefficient of determination: 0.3648848114133756
intercept: [4.21227661]
slope: [[ 0.          0.16642726 -0.17124379  0.01709362  0.00499221]]
predicted response:
[[4.54638371]
 [6.0085059 ]
 [6.18866689]
 [3.02076176]
 [3.78154498]
 [5.74605327]
 [4.27525042]
 [2.8380248 ]
 [6.59480827]]


In [47]:
#https://scikit-learn.org/stable/modules/svm.html
#https://scikit-learn.org/stable/auto_examples/svm/plot_svm_regression.html
from sklearn import svm
regr = svm.SVR()
regr.fit(X, Y)
regr.predict(X)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


array([4.6201865 , 4.89999999, 4.35957831, 3.91903191, 4.63619573,
       4.26652356, 4.09999998, 3.64670873, 4.88017073])